In [5]:
import cv2
import pandas as pd
import numpy as np
from csv import *
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import os

In [61]:
from keras.layers import concatenate
from keras.layers import Input
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import MaxPooling2D, Flatten,Conv2D, Dense,BatchNormalization,GlobalAveragePooling2D,Dropout
from keras.applications.densenet import DenseNet121
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.vgg16 import VGG16
from keras.applications.resnet_v2 import ResNet50V2
from keras.applications.inception_v3 import InceptionV3
from keras.models import load_model

In [7]:
train_csv = r"C:\Users\ABHINAV\Downloads\train-20230326T213916Z-001\train\Ground_Truth.csv"
train_img = r"C:\Users\ABHINAV\Downloads\train-20230326T213916Z-001\train\train images"
test_img = r"C:\Users\ABHINAV\Downloads\train-20230326T213916Z-001\test_images-20230331T075032Z-001\test_images"

In [8]:
lis = (os.listdir(r"C:\Users\ABHINAV\Downloads\train-20230326T213916Z-001\train\train images"))

In [24]:
def image_equalization(image):
    
    R, G, B = cv2.split(image)
    output1_R = cv2.equalizeHist(R)
    output1_G = cv2.equalizeHist(G)
    output1_B = cv2.equalizeHist(B)
    equ = cv2.merge((output1_R, output1_G, output1_B))
    
    return equ

In [ ]:
y  = []
y1 = []
df = pd.read_csv(train_csv)
for i in range(len(lis)):
    if df.iloc[i][0] in lis:
        if df.iloc[i][1] == "No Finding":
            y.append(0)
        else:
            y.append(1)

        img_arr = cv2.imread(train_img+chr(92)+df.iloc[i][0])

        resized_img = cv2.resize(img_arr, (224, 224),interpolation = cv2.INTER_CUBIC)
        #print(resized_img.shape)
        eq_img = image_equalization(resized_img)

        y1.append(eq_img.astype(np.float16))
    


In [239]:
count_ap = 0
count_disease = 0 # given in ap, if person has any disease
for i in range(df.shape[0]):
    if df.iloc[i][-1] == "PA":
        count_ap +=1
        if df.iloc[i][1] != "No Finding":
            count_disease += 1
print(count_disease/count_ap)

0.41431875525651807


In [26]:
X_train, X_val, y_train, y_val = train_test_split(y1, y, test_size=0.2, random_state=42)

In [29]:
X_train = np.array(X_train)

In [30]:
X_val = np.array(X_val)

In [31]:
y_train = np.array(y_train)
y_val = np.array(y_val)

In [32]:
X_train/=255

X_val/=255

In [33]:
print(len(y_train))
print(X_train.shape)

3185
(3185, 224, 224, 3)


not now

not now

In [91]:
cnn = keras.models.Sequential([ 

#conv + pool layer 1
keras.layers.Conv2D(filters = 32, kernel_size=(4,4), activation = 'leaky_relu',  input_shape=(512, 512, 1)),
keras.layers.MaxPooling2D((2,2)), 
# conv + pool layer 2
keras.layers.Conv2D(filters = 64, kernel_size=(4,4), activation = 'leaky_relu', input_shape=(512, 512, 1)),
keras.layers.MaxPooling2D((2,2)),

# dense
keras.layers.Flatten(),
keras.layers.Dense(128, activation = 'leaky_relu'),
keras.layers.Dropout(0.25),
keras.layers.Dense(64, activation = 'leaky_relu'),
keras.layers.Dropout(0.25),
keras.layers.BatchNormalization(),
keras.layers.Dense(1, activation='softmax')
])
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [65]:
print(X_train.shape)

(3185, 224, 224, 3)


In [66]:
model_1 = load_model(r"C:\Users\ABHINAV\Downloads\train-20230326T213916Z-001\CheXpert_DenseNet121_res224.h5")
out1 = model_1.predict(X_train)

100/100 [==============================] - 231s 2s/step


In [77]:
def get_y(arr):
    max = -1
    id = 0
    for i in range(arr.shape[0]):
        if arr[i]>max:
            max = arr[i]
            id = i
    if id == 0:
        return 0
    else:
        return 1



In [80]:
print(y_train[4])
print((out1[4]))
print(get_y(out1[4]))

1
[1.0331949e-11 5.2967995e-02 4.0879318e-01 9.6949965e-01 3.5778505e-03
 8.9986980e-07 3.8322914e-02 4.1742545e-02 9.9209678e-01 4.1210537e-08
 1.0000000e+00 9.4147184e-13 1.8464064e-02 1.8471157e-03]
1


In [52]:
# resnet

input_layer = Input(shape = (224, 224, 3))


res_base = ResNet50V2(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

for layer in res_base.layers:
    layer.trainable = False

model_res = res_base(input_layer)
model_res = GlobalAveragePooling2D()(model_res)
output_res = Flatten()(model_res)

x = BatchNormalization()(output_res)
x = Dense(256,activation = "relu")(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(128,activation = "relu")(x)
x = Dropout(0.5)(x)
x = Dense(1, activation = "sigmoid")(x)
resnet = tf.keras.models.Model(inputs = input_layer, outputs = x)
resnet.compile(optimizer = "Adam", loss = 'binary_crossentropy', metrics = ['accuracy'])


In [ ]:
from keras.models import load_model

model = load_model(r'C:\Users\ABHINAV\Downloads\train-20230326T213916Z-001\cheXnet.h5')

# Remove the weights from the last layer
model.layers[-1].set_weights([np.zeros_like(w) for w in model.layers[-1].get_weights()])

# Save the modified model
model.save('my_modified_model.h5')

In [ ]:
# densenet over cheXnet

input_layer = Input(shape = (224, 224, 3))

densenet_base = DenseNet121(input_shape=(224, 224, 3), weights=r'C:\Users\ABHINAV\Downloads\train-20230326T213916Z-001\cheXnet.h5')

for layer in densenet_base.layers:
    layer.trainable = False

model_densenet = densenet_base(input_layer)
model_densenet = GlobalAveragePooling2D()(model_densenet)
output_densenet = Flatten()(model_densenet)

x = BatchNormalization()(output_densenet)
x = Dense(256,activation = "relu")(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(128,activation = "relu")(x)
x = Dropout(0.5)(x)
x = Dense(1, activation = "sigmoid")(x)
densenet = tf.keras.models.Model(inputs = input_layer, outputs = x)
densenet.compile(optimizer = "Adam", loss = 'binary_crossentropy', metrics = ['accuracy'])




In [189]:
# google net

input_layer = Input(shape = (224, 224, 3))

google_base = InceptionV3(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

for layer in google_base.layers:
    layer.trainable = False

model_google = res_base(input_layer)
model_google = GlobalAveragePooling2D()(model_google)
output_google = Flatten()(model_google)

x = BatchNormalization()(output_google)
x = Dense(256,activation = "relu")(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(128,activation = "relu")(x)
x = Dropout(0.5)(x)
x = Dense(1, activation = "sigmoid")(x)
googlenet = tf.keras.models.Model(inputs = input_layer, outputs = x)
googlenet.compile(optimizer = "Adam", loss = 'binary_crossentropy', metrics = ['accuracy'])

87910968/87910968 [==============================] - 28s 0us/step


In [192]:
# mobile net

input_layer = Input(shape = (224, 224, 3))

mobilenet_base = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

for layer in mobilenet_base.layers:
    layer.trainable = False

model_mobilenet = mobilenet_base(input_layer)
model_mobilenet = GlobalAveragePooling2D()(model_mobilenet)
output_mobilenet = Flatten()(model_mobilenet)

x = BatchNormalization()(output_mobilenet)
x = Dense(256,activation = "relu")(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(128,activation = "relu")(x)
x = Dropout(0.5)(x)
x = Dense(1, activation = "sigmoid")(x)
mobilenet = tf.keras.models.Model(inputs = input_layer, outputs = x)
mobilenet.compile(optimizer = "Adam", loss = 'binary_crossentropy', metrics = ['accuracy'])

In [193]:
#vgg

input_layer = Input(shape = (224, 224, 3))

vgg_base = VGG16(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

for layer in vgg_base.layers:
    layer.trainable = False
    
model_vgg = vgg_base(input_layer)
model_vgg = GlobalAveragePooling2D()(model_vgg)
output_vgg = Flatten()(model_vgg)


x = BatchNormalization()(output_vgg)
x = Dense(256,activation = "relu")(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(128,activation = "relu")(x)
x = Dropout(0.5)(x)
x = Dense(1, activation = "sigmoid")(x)
vggnet = tf.keras.models.Model(inputs = input_layer, outputs = x)
vggnet.compile(optimizer = "Adam", loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
#densenet

input_layer = Input(shape = (224, 224, 3))

densenet_base = InceptionV3(input_shape=(224, 224, 3), include_top=False, weights=)

for layer in densenet_base.layers:
    layer.trainable = False

model_densenet = densenet_base(input_layer)
model_densenet = GlobalAveragePooling2D()(model_densenet)
output_densenet = Flatten()(model_densenet)

x = BatchNormalization()(output_densenet)
x = Dense(256,activation = "relu")(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(128,activation = "relu")(x)
x = Dropout(0.5)(x)
x = Dense(1, activation = "sigmoid")(x)
densenet = tf.keras.models.Model(inputs = input_layer, outputs = x)
densenet.compile(optimizer = "Adam", loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
import torchxrayvision as xrv
import skimage, torch, torchvision

model = xrv.models.DenseNet(weights="densenet121-res224-nih")

In [174]:
# combined model
input_layer = Input(shape = (256, 256, 3))
#first model
mobilenet_base = MobileNetV2(input_shape=(256, 256, 3), include_top=False, weights="imagenet")

densenet_base = DenseNet169(input_shape=(256, 256, 3), include_top=False, weights="imagenet")

vgg_base = VGG16(input_shape=(256, 256, 3), include_top=False, weights="imagenet")

for layer in mobilenet_base.layers:
    layer.trainable =  False
for layer in densenet_base.layers:
    layer.trainable = False
for layer in vgg_base.layers:
    layer.trainable = False
    
model_mobilenet = mobilenet_base(input_layer)
model_mobilenet = GlobalAveragePooling2D()(model_mobilenet)
output_mobilenet = Flatten()(model_mobilenet)
model_densenet = densenet_base(input_layer)
model_densenet = GlobalAveragePooling2D()(model_densenet)
output_densenet = Flatten()(model_densenet)
model_vgg = densenet_base(input_layer)
model_vgg = GlobalAveragePooling2D()(model_vgg)
output_vgg = Flatten()(model_vgg)

merged = tf.keras.layers.Concatenate()([output_mobilenet, output_densenet,output_vgg])

x = BatchNormalization()(merged)
x = Dense(256,activation = "relu")(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(128,activation = "relu")(x)
x = Dropout(0.5)(x)
x = Dense(1, activation = "sigmoid")(x)
stacked_model = tf.keras.models.Model(inputs = input_layer, outputs = x)
stacked_model.compile(optimizer = "Adam", loss = 'binary_crossentropy', metrics = ['accuracy'])

58889256/58889256 [==============================] - 13s 0us/step


In [34]:
#densenet121+resnet+googlenet

input_layer = Input(shape = (224, 224, 3))
#first model
googlenet_base = InceptionV3(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

densenet_base = DenseNet121(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

resnet_base = ResNet50V2(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

for layer in googlenet_base.layers:
    layer.trainable =  False
for layer in densenet_base.layers:
    layer.trainable = False
for layer in resnet_base.layers:
    layer.trainable = False
    
model_googlenet = googlenet_base(input_layer)
model_googlenet = GlobalAveragePooling2D()(model_googlenet)
output_googlenet = Flatten()(model_googlenet)
model_densenet = densenet_base(input_layer)
model_densenet = GlobalAveragePooling2D()(model_densenet)
output_densenet = Flatten()(model_densenet)
model_resnet = resnet_base(input_layer)
model_resnet = GlobalAveragePooling2D()(model_resnet)
output_resnet = Flatten()(model_resnet)

merged = tf.keras.layers.Concatenate()([output_googlenet, output_densenet,output_resnet])

x = BatchNormalization()(merged)
x = Dense(256,activation = "relu")(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(128,activation = "relu")(x)
x = Dropout(0.5)(x)
x = Dense(1, activation = "sigmoid")(x)
stacked_model = tf.keras.models.Model(inputs = input_layer, outputs = x)
stacked_model.compile(optimizer = "Adam", loss = 'binary_crossentropy', metrics = ['accuracy'])



29084464/29084464 [==============================] - 8s 0us/step


In [35]:
stacked_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 inception_v3 (Functional)      (None, 5, 5, 2048)   21802784    ['input_5[0][0]']                
                                                                                                  
 densenet121 (Functional)       (None, 7, 7, 1024)   7037504     ['input_5[0][0]']                
                                                                                                  
 resnet50v2 (Functional)        (None, 7, 7, 2048)   23564800    ['input_5[0][0]']            

In [36]:
from keras.callbacks import ModelCheckpoint

model_save = ModelCheckpoint('./stacked_model.h5',
save_best_only = True,
save_weights_only = False,
monitor = 'val_loss', 
mode = 'min', verbose = 1)

In [37]:
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_accuracy', patience=3, verbose=1)
mcp_save = ModelCheckpoint('./sack1net.hd5',save_best_only=True,monitor='val_accuracy',mode='max')
stacked_model.fit(X_train, y_train,epochs=10, validation_data=(X_val, y_val), callbacks=[early_stop, mcp_save])

Epoch 1/10
100/100 [==============================] - ETA: 0s - loss: 0.7785 - accuracy: 0.5928

INFO:tensorflow:Assets written to: .\sack1net.hd5\assets


INFO:tensorflow:Assets written to: .\sack1net.hd5\assets


100/100 [==============================] - 727s 7s/step - loss: 0.7785 - accuracy: 0.5928 - val_loss: 0.6511 - val_accuracy: 0.6286
Epoch 2/10
100/100 [==============================] - ETA: 0s - loss: 0.6485 - accuracy: 0.6496

INFO:tensorflow:Assets written to: .\sack1net.hd5\assets


...

INFO:tensorflow:Assets written to: .\sack1net.hd5\assets


100/100 [==============================] - 726s 7s/step - loss: 0.5037 - accuracy: 0.7592 - val_loss: 0.6565 - val_accuracy: 0.6462
Epoch 7/10
100/100 [==============================] - ETA: 0s - loss: 0.4717 - accuracy: 0.7730

INFO:tensorflow:Assets written to: .\sack1net.hd5\assets


INFO:tensorflow:Assets written to: .\sack1net.hd5\assets


100/100 [==============================] - 761s 8s/step - loss: 0.4717 - accuracy: 0.7730 - val_loss: 0.6820 - val_accuracy: 0.6474
Epoch 8/10
100/100 [==============================] - 613s 6s/step - loss: 0.4347 - accuracy: 0.8000 - val_loss: 0.6965 - val_accuracy: 0.6424
Epoch 9/10
100/100 [==============================] - 608s 6s/step - loss: 0.4033 - accuracy: 0.8173 - val_loss: 0.7652 - val_accuracy: 0.6336
Epoch 10/10
100/100 [==============================] - 618s 6s/step - loss: 0.3744 - accuracy: 0.8377 - val_loss: 0.7737 - val_accuracy: 0.6261
Epoch 10: early stopping


In [54]:
from keras.models import load_model

google = load_model(r"C:\Users\ABHINAV\Downloads\train-20230326T213916Z-001\googlenet.hd5")

In [176]:
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
stacked_model.fit(X_train, y_train,epochs=10, validation_data=(X_val, y_val), callbacks=[early_stop, model_save])


Epoch 1/10
100/100 [==============================] - ETA: 0s - loss: 0.8113 - accuracy: 0.5802
Epoch 1: val_loss improved from inf to 0.63399, saving model to .\stacked_model.h5
100/100 [==============================] - 1033s 10s/step - loss: 0.8113 - accuracy: 0.5802 - val_loss: 0.6340 - val_accuracy: 0.6487
Epoch 2/10
100/100 [==============================] - ETA: 0s - loss: 0.6795 - accuracy: 0.6546
Epoch 2: val_loss improved from 0.63399 to 0.63013, saving model to .\stacked_model.h5
100/100 [==============================] - 989s 10s/step - loss: 0.6795 - accuracy: 0.6546 - val_loss: 0.6301 - val_accuracy: 0.6600
Epoch 3/10
100/100 [==============================] - ETA: 0s - loss: 0.6337 - accuracy: 0.6703
Epoch 3: val_loss improved from 0.63013 to 0.62387, saving model to .\stacked_model.h5
100/100 [==============================] - 964s 10s/step - loss: 0.6337 - accuracy: 0.6703 - val_loss: 0.6239 - val_accuracy: 0.6587
Epoch 4/10
100/100 [==============================] - E

In [ ]:
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_accuracy', patience=3, verbose=1)
mcp_save = ModelCheckpoint('./densenet.hd5',save_best_only=True,monitor='val_accuracy',mode='max')
densenet.fit(X_train, y_train,epochs=10, validation_data=(X_val, y_val), callbacks=[early_stop, mcp_save])

In [ ]:
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_accuracy', patience=3, verbose=1)
mcp_save = ModelCheckpoint('./resnet.hd5',save_best_only=True,monitor='val_accuracy',mode='max')
resnet.fit(X_train, y_train,epochs=10, validation_data=(X_val, y_val), callbacks=[early_stop, mcp_save])

In [218]:
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_accuracy', patience=3, verbose=1)
mcp_save = ModelCheckpoint('./mobnet.hd5',save_best_only=True,monitor='val_accuracy',mode='max')
mobilenet.fit(X_train, y_train,epochs=10, validation_data=(X_val, y_val), callbacks=[early_stop,mcp_save])

Epoch 1/10
98/98 [==============================] - ETA: 0s - loss: 0.7075 - accuracy: 0.6194

INFO:tensorflow:Assets written to: .\mobnet.hd5\assets


INFO:tensorflow:Assets written to: .\mobnet.hd5\assets


98/98 [==============================] - 106s 1s/step - loss: 0.7075 - accuracy: 0.6194 - val_loss: 0.6206 - val_accuracy: 0.6718
Epoch 2/10
98/98 [==============================] - 86s 880ms/step - loss: 0.6653 - accuracy: 0.6353 - val_loss: 0.6253 - val_accuracy: 0.6718
Epoch 3/10
98/98 [==============================] - ETA: 0s - loss: 0.6328 - accuracy: 0.6632

INFO:tensorflow:Assets written to: .\mobnet.hd5\assets


INFO:tensorflow:Assets written to: .\mobnet.hd5\assets


98/98 [==============================] - 108s 1s/step - loss: 0.6328 - accuracy: 0.6632 - val_loss: 0.6211 - val_accuracy: 0.6743
Epoch 4/10
98/98 [==============================] - 89s 906ms/step - loss: 0.6179 - accuracy: 0.6676 - val_loss: 0.6175 - val_accuracy: 0.6679
Epoch 5/10
98/98 [==============================] - 88s 900ms/step - loss: 0.5790 - accuracy: 0.7021 - val_loss: 0.6323 - val_accuracy: 0.6718
Epoch 6/10
98/98 [==============================] - 92s 938ms/step - loss: 0.5497 - accuracy: 0.7204 - val_loss: 0.6472 - val_accuracy: 0.6539
Epoch 6: early stopping


In [224]:
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_accuracy', patience=3, verbose=1)
mcp_save = ModelCheckpoint('./mobnet.hd5',save_best_only=True,monitor='val_accuracy',mode='max')
vggnet.fit(X_train, y_train,epochs=10, validation_data=(X_val, y_val), callbacks=[early_stop,mcp_save])

Epoch 1/10
98/98 [==============================] - ETA: 0s - loss: 0.8108 - accuracy: 0.5948

INFO:tensorflow:Assets written to: .\mobnet.hd5\assets


INFO:tensorflow:Assets written to: .\mobnet.hd5\assets


98/98 [==============================] - 706s 7s/step - loss: 0.8108 - accuracy: 0.5948 - val_loss: 0.6823 - val_accuracy: 0.5964
Epoch 2/10
98/98 [==============================] - 716s 7s/step - loss: 0.7253 - accuracy: 0.6024 - val_loss: 0.6885 - val_accuracy: 0.5287
Epoch 3/10
98/98 [==============================] - ETA: 0s - loss: 0.6963 - accuracy: 0.6200

INFO:tensorflow:Assets written to: .\mobnet.hd5\assets


...

INFO:tensorflow:Assets written to: .\mobnet.hd5\assets


98/98 [==============================] - 612s 6s/step - loss: 0.6202 - accuracy: 0.6657 - val_loss: 0.6129 - val_accuracy: 0.6692
Epoch 7/10
98/98 [==============================] - 604s 6s/step - loss: 0.6154 - accuracy: 0.6683 - val_loss: 0.6132 - val_accuracy: 0.6654
Epoch 8/10
98/98 [==============================] - ETA: 0s - loss: 0.6217 - accuracy: 0.6791

INFO:tensorflow:Assets written to: .\mobnet.hd5\assets


INFO:tensorflow:Assets written to: .\mobnet.hd5\assets


98/98 [==============================] - 610s 6s/step - loss: 0.6217 - accuracy: 0.6791 - val_loss: 0.6051 - val_accuracy: 0.6743
Epoch 9/10
98/98 [==============================] - 601s 6s/step - loss: 0.5963 - accuracy: 0.6785 - val_loss: 0.6085 - val_accuracy: 0.6743
Epoch 10/10
98/98 [==============================] - 602s 6s/step - loss: 0.5885 - accuracy: 0.6938 - val_loss: 0.6044 - val_accuracy: 0.6641


In [230]:
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_accuracy', patience=3, verbose=1)
mcp_save = ModelCheckpoint('./googlenet.hd5',save_best_only=True,monitor='val_accuracy',mode='max')
googlenet.fit(X_train, y_train,epochs=10, validation_data=(X_val, y_val), callbacks=[early_stop,mcp_save])

Epoch 1/10
98/98 [==============================] - ETA: 0s - loss: 0.8260 - accuracy: 0.5839

INFO:tensorflow:Assets written to: .\googlenet.hd5\assets


INFO:tensorflow:Assets written to: .\googlenet.hd5\assets


98/98 [==============================] - 248s 2s/step - loss: 0.8260 - accuracy: 0.5839 - val_loss: 0.6195 - val_accuracy: 0.6539
Epoch 2/10
98/98 [==============================] - ETA: 0s - loss: 0.7078 - accuracy: 0.6168

INFO:tensorflow:Assets written to: .\googlenet.hd5\assets


...

INFO:tensorflow:Assets written to: .\googlenet.hd5\assets


98/98 [==============================] - 258s 3s/step - loss: 0.6514 - accuracy: 0.6590 - val_loss: 0.6051 - val_accuracy: 0.6833
Epoch 4/10
98/98 [==============================] - ETA: 0s - loss: 0.6152 - accuracy: 0.6740

INFO:tensorflow:Assets written to: .\googlenet.hd5\assets


INFO:tensorflow:Assets written to: .\googlenet.hd5\assets


98/98 [==============================] - 265s 3s/step - loss: 0.6152 - accuracy: 0.6740 - val_loss: 0.6118 - val_accuracy: 0.6845
Epoch 5/10
98/98 [==============================] - 244s 2s/step - loss: 0.5877 - accuracy: 0.6983 - val_loss: 0.6114 - val_accuracy: 0.6731
Epoch 6/10
98/98 [==============================] - 247s 3s/step - loss: 0.5520 - accuracy: 0.7290 - val_loss: 0.6198 - val_accuracy: 0.6616
Epoch 7/10
98/98 [==============================] - 248s 3s/step - loss: 0.5331 - accuracy: 0.7370 - val_loss: 0.6351 - val_accuracy: 0.6705
Epoch 7: early stopping


In [42]:
lis2 = os.listdir(test_img)
X_test = []
for item in lis2:
    img = cv2.imread(test_img+chr(92)+item)
    resized_img = cv2.resize(img, (224, 224), interpolation = cv2.INTER_CUBIC)
    X_test.append(resized_img.astype(np.float16))
    

In [43]:
print(len(X_test))

998


In [44]:
X_test = np.array(X_test)
X_test/=255

In [246]:
y_out = stacked_model.predict(X_test)


32/32 [==============================] - 168s 5s/step


In [55]:
y_out = google.predict(X_test)

32/32 [==============================] - 57s 2s/step


In [81]:
y_out1 = model_1.predict(X_test)

32/32 [==============================] - 81s 3s/step


In [84]:
opt_wts = []
for i in range(y_out1.shape[0]):
    opt_wts.append(get_y(y_out1[i]))

In [85]:
print(opt_wts)

[1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 

In [56]:
print(y_out.shape)
print(len(lis2))

(998, 1)
998


In [57]:
lis3 = []
for i in range(y_out.shape[0]):
    if y_out[i] < 0.5:
        lis3.append(0)
    else:
        lis3.append(1)

In [58]:
print(sum(lis3))

462


In [86]:
df2 = pd.DataFrame()
df2["Image Index"] = lis2
df2["Finding Labels"] = opt_wts



In [87]:
df2.to_csv("opt2.csv", index=False)